# 확률적 경사 하강법
경사 하강법 알고리즘 이해, 대량으 ㅣ데이터에서 분류 모델을 훈련하는 방법 습득

이전에 훈련한 모델을 버리지 않고 새로운 데이터에 대해서만 조금씩 더 훈련하는 것

***점진적 학습***

대표적인 점진적 학습에는 ***확률적 경사 하강법***이 있음

경사 하강법
- 가장 가파른 경사를 따라 원하는 지점에 도달하는 것이 목표

***확률적***에 대해
- 훈련 세트를 사용하여 가장 가파른 길을 찾을 것
- 전체 샘플을 사용하지 않고 딱 하나의 샘플을 훈련 세트에서 랜덤하게 골라 가장 가파른 길을 찾는다
- 훈련 세트에서 랜덤하게 하나의 샘플을 고르는 것이 바로 ***확률적 경사 하강법***

***에포크***
- 확률적 경사 하강법에서 훈련 세트를 한 번 모두 사용하는 과정

***미니배치 경사 하강법***
- 여러 개의 샘플을 사용해 경사 하강법을 수행하는 방식

***배치 경사 하강법***
- 극단적으로 한 번 경사로를 따라 이동하기 위해 전체 샘플을 사용하는 것

***신경망 알고리즘***은 확률적 경사 하강법을 꼭 사용하는 알고리즘

***손실 함수***는 어떤 문제에서 머신러닝 알고리즘이 얼마나 엉터리인지 측정하는 기준
- 손실 함수의 값은 작을 수록 좋음

기술적으로 손실 함수는 미분적(연속적)이어야 함

***로지스틱 손실 함수***
- 양성 클래스(타깃 = 1)일 때 손실은 $-log(예측 확률)$로 계산
    - 확률이 1에서 멀어져 0에 가까워질수록 손실은 아주 큰 양수가 됨
- 음성 클래스(타깃 = 0)일 때 손실은 $-log(1-예측 확률)$로 계산
    - 예측 확률이 0에서 멀어져 1에 가까워질수록 손실은 아주 큰 양수가 됨

***이진 크로스엔트로피 손실 함수***라고도 함

다중 분류에서 사용하는 손실함수는 ***크로스엔트로피 손실 함수***

In [ ]:
# 데이터 불러오기
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')

In [ ]:
# Species 열을 제외한 나머지 5개는 입력 데이터로 사용
# Species 열은 타깃 데이터
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
fish_target = fish['Species'].to_numpy()

In [ ]:
# 사이킷런의 train_test_split() 함수를 사용하여 이 데이터를 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

In [ ]:
# 훈련 세트와 테스트 세트의 특성을 표준화 전처리
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [ ]:
# 사이킷런에서 제공하는 대표적인 확률적 경사 하강법 클래스는 SGDClassfier
from sklearn.linear_model import SGDClassifier

In [ ]:
# 객체 생성 시 2개의 매개변수 지정
# loss : 손실 함수의 종류
# max_iter : 수행할 에포크 횟수
sc = SGDClassifier(loss='log_loss', max_iter=10, random_state=42)
sc.fit(train_scaled, train_target)
print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

In [ ]:
# 점진적인 학습이 가능
# 객체를 다시 만들지 않고 모델 sc를 추가로 더 훈련
# 이어서 훈련할 시 partial_fit() 메서드 사용
sc.partial_fit(train_scaled, train_target)
print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

확률적 경사 하강법 모델은  에포크 횟수에 따라 과소적합이나 과대적합이 될 수 있다
- 적은 에포크 횟수 동안 훈련한 모델은 훈련 세트와 테스트 세트에 잘 맞지 않는 과소적합된 모델일 가능성이 높다
- 많은 에포크 횟수 동안 훈련한 모델은 훈련 세트에 너무 잘 맞아 테스트 세트에는 오히려 점수가 나쁜 과대적합이 될 가능성이 있음

과대적합이 시작하기 전 훈련을 멈추는 것을 ***조기 종료***라고 함


In [ ]:
# partial_fit()메서드만 사용하려면 훈련 세트에 있는 전체 클래스의 레이블을 partial_fit() 메서드에 전달해 줘야 함
# np.unique() 함수로 train_target에 있는 7개 생선의 목록을 생성
# 에포크마다 훈련 세트와 테스트 세트에 대한 점수를 기록하기 위한 2개의 리스트 생성
import numpy as np
sc = SGDClassifier(loss='log_loss', random_state=42)
train_score = []
test_score = []

classes = np.unique(train_target)

In [ ]:
# 300번의 에포크 동안 훈련을 반복하여 진행
# 반복마다 훈련 세트와 테스트 세트의 점수를 계산하여 리스트에 추가
for _ in range(0, 300):
    sc.partial_fit(train_scaled, train_target, classes=classes)
    train_score.append(sc.score(train_scaled, train_target))
    test_score.append(sc.score(test_scaled, test_target))

In [ ]:
# 300번의 에포크 동안 기록한 점수를 그래프로 플롯
import matplotlib.pyplot as plt
plt.plot(train_score)
plt.plot(test_score)
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [ ]:
# 반복 횟수를 100에 맞추고 모델 다시 훈련
sc = SGDClassifier(loss='log_loss', max_iter=100, tol=None, random_state=42)
sc.fit(train_scaled, train_target)
print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

SGDClassifier는 일정 에포크 동안 성능이 향상되지 않으면 더 훈련하지 않고 자동으로 멈춤

tol 매개변수에서 향상될 최솟값을 지정
- tol을 None으로 지정하여 자동으로 멈추지 않고 max_iter=100만큼 무조건 반복하도록 함

loss 매개변수의 기본값은 'hinge'

***힌지 손실***은 ***서포트 벡터 머신***이라 불리는 다른 머신러닝 알고리즘을 위한 손실 함수

In [ ]:
# 힌지 손실을 사용한 모델
sc = SGDClassifier(loss='hinge', max_iter=100, tol=None, random_state=42)
sc.fit(train_scaled, train_target)
print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))